In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import odym.classes as msc
import odym.functions as msf
import odym.dynamic_stock_model as dsm
import os

In [95]:
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), "data")

df_composition = pd.read_excel(os.path.join(DATA_PATH, "clustered_fiber_composition.xlsx"))

In [96]:
clothing_type = list(df_composition["Clothing type"])
clothing_type

['t-shirts',
 'trousers',
 'overcoats',
 'underwear',
 'handkerchiefs_1',
 'shirts_1',
 'sportswear',
 'handkerchiefs_2',
 'shirts_2',
 'sweaters']

In [97]:
meta_columns = ['Category', 'Sum', 'Cluster', 'Count', 'Market share', 'Category share', 'Lifetime Min', 'Lifetime Max', 'Clothing type', 'Fibre composition sum']
fibers = df_composition.columns[~df_composition.columns.isin(meta_columns)].tolist()
fibers

['Acrylic',
 'Cotton',
 'Polyamide/nylon',
 'Polyester',
 'Silk',
 'Viscose',
 'Wool',
 'Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)']

In [98]:
quarters = [f"{year}Q{q}" for year in range(2016, 2023) for q in range(1, 5)]
quarters

['2016Q1',
 '2016Q2',
 '2016Q3',
 '2016Q4',
 '2017Q1',
 '2017Q2',
 '2017Q3',
 '2017Q4',
 '2018Q1',
 '2018Q2',
 '2018Q3',
 '2018Q4',
 '2019Q1',
 '2019Q2',
 '2019Q3',
 '2019Q4',
 '2020Q1',
 '2020Q2',
 '2020Q3',
 '2020Q4',
 '2021Q1',
 '2021Q2',
 '2021Q3',
 '2021Q4',
 '2022Q1',
 '2022Q2',
 '2022Q3',
 '2022Q4']

In [99]:
df_production = pd.read_excel(os.path.join(DATA_PATH, "clothing_production_interpolated.xlsx"), index_col=0)

# Split each year's value into 4 quarters (equal distribution)
quarters = []
for year in df_production.columns:
    for q in range(1, 5):
        quarters.append(f"{year}Q{q}")

df_production_quarterly = pd.DataFrame(
    np.repeat(df_production.values / 4, 4, axis=1),
    index=df_production.index,
    columns=quarters
)
# Remove the 'total' row if present
df_production_quarterly = df_production_quarterly.drop('total', errors='ignore')
df_production_quarterly

,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,...,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4
Product category,,,,,,,,,,,,,,,,,,,,,
overcoats and anoraks,1198.25,1198.25,1198.25,1198.25,1425.25,1425.25,1425.25,1425.25,1652.00,1652.00,...,2106.00,2106.00,2332.75,2332.75,2332.75,2332.75,2559.75,2559.75,2559.75,2559.75
suits and blazers,799.50,799.50,799.50,799.50,879.25,879.25,879.25,879.25,959.00,959.00,...,1118.50,1118.50,1198.25,1198.25,1198.25,1198.25,1278.00,1278.00,1278.00,1278.00
trousers and shorts,3479.75,3479.75,3479.75,3479.75,3926.50,3926.50,3926.50,3926.50,4373.50,4373.50,...,5267.25,5267.25,5714.00,5714.00,5714.00,5714.00,6161.00,6161.00,6161.00,6161.00
dresses and skirts,676.75,676.75,676.75,676.75,784.75,784.75,784.75,784.75,892.75,892.75,...,1108.75,1108.75,1216.75,1216.75,1216.75,1216.75,1324.75,1324.75,1324.75,1324.75
"shirts, blouses, tops",1437.50,1437.50,1437.50,1437.50,1371.00,1371.00,1371.00,1371.00,1304.75,1304.75,...,1172.00,1172.00,1105.50,1105.50,1105.50,1105.50,1039.25,1039.25,1039.25,1039.25
"underwear, socks, and night clothes",1784.00,1784.00,1784.00,1784.00,1773.75,1773.75,1773.75,1773.75,1763.50,1763.50,...,1742.75,1742.75,1732.50,1732.50,1732.50,1732.50,1722.25,1722.25,1722.25,1722.25
t-shirts and vests,2262.00,2262.00,2262.00,2262.00,2338.75,2338.75,2338.75,2338.75,2415.50,2415.50,...,2568.75,2568.75,2645.50,2645.50,2645.50,2645.50,2722.25,2722.25,2722.25,2722.25
sweaters and cardigans,2282.50,2282.50,2282.50,2282.50,2514.75,2514.75,2514.75,2514.75,2747.00,2747.00,...,3211.25,3211.25,3443.50,3443.50,3443.50,3443.50,3675.75,3675.75,3675.75,3675.75
sportswear and swimwear,335.75,335.75,335.75,335.75,879.75,879.75,879.75,879.75,1423.75,1423.75,...,2512.00,2512.00,3056.00,3056.00,3056.00,3056.00,3600.00,3600.00,3600.00,3600.00


In [100]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def compute_survival_matrix(df_composition, quarters):
    min_life = df_composition['Lifetime Min'].values * 4  # in quarters
    max_life = df_composition['Lifetime Max'].values * 4  # in quarters

    mu = (min_life + max_life) / 2
    sigma = np.maximum((max_life - min_life) / 4, 1.0)

    x = np.arange(len(quarters))
    prob_matrix = norm.pdf(x[None, :], loc=mu[:, None], scale=sigma[:, None])

    # Handle zero-variance edge case: replace rows where sigma == 0 with Dirac delta
    is_delta = sigma == 0
    if np.any(is_delta):
        prob_matrix[is_delta] = 0
        peak_positions = mu[is_delta].astype(int).clip(0, len(x)-1)
        prob_matrix[is_delta, peak_positions] = 1

    # Normalize rows to sum to 1
    prob_matrix /= prob_matrix.sum(axis=1, keepdims=True)
    survival_matrix = 1 - np.cumsum(prob_matrix, axis=1)

    # Ensure numerical stability
    survival_matrix[:, 0] = 1.0
    survival_matrix[:, -1] = 0.0
    survival_matrix = np.clip(survival_matrix, 0.0, 1.0)

    df_survival = pd.DataFrame(survival_matrix, columns=x)
    df_survival.insert(0, "Clothing type", types)
    return survival_matrix

# Example usage:
survival_matrix = compute_survival_matrix(df_composition, quarters)



In [101]:
ModelClassification  = {} # Create dictionary of model classifications

ModelClassification['Time'] = msc.Classification(
    Name='Time',
    Dimension='Time',
    ID=1,
    Items=range(len(quarters))

)

ModelClassification['Age_Cohort'] = msc.Classification(
    Name='Age Cohort',
    Dimension='Time',
    ID=2,
    Items=ModelClassification['Time'].Items,
)

ModelClassification['Element'] = msc.Classification(
    Name='Fibers',
    Dimension='Element',
    ID=3,
    Items= fibers,
)

ModelClassification["Clothing"] = msc.Classification(
    Name='Clothing Type',
    Dimension='Clothing',
    ID=4,
    Items = clothing_type,
)   

Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End   = int(max(ModelClassification['Time'].Items))
Model_Duration   = Model_Time_End - Model_Time_Start

In [102]:
IndexTable = pd.DataFrame({'Aspect'        : ['Time', 'Age_Cohort', 'Element','Clothing'], # 'Time' and 'Element' must be present!
                           'Description'   : ['Model aspect "time"', 'Model aspect "age cohort"','Model aspect "fiber type"', 'Model aspect "clothing type"'],
                           'Dimension'     : ['Time','Time','Element','Clothing'], # 'Time' and 'Element' are also dimensions
                           'Classification': [ModelClassification[Aspect] for Aspect in ['Time','Age_Cohort','Element', 'Clothing']],
                           'IndexLetter'   : ['t','a','e','c']}) # Unique one letter (upper or lower case) indices to be used later for calculations.

IndexTable.set_index('Aspect', inplace = True) # Default indexing of IndexTable, other indices are produced on the fly

IndexTable

,Description,Dimension,Classification,IndexLetter
Aspect,,,,
Time,"Model aspect ""time""",Time,<odym.classes.classification.Classification ob...,t
Age_Cohort,"Model aspect ""age cohort""",Time,<odym.classes.classification.Classification ob...,a
Element,"Model aspect ""fiber type""",Element,<odym.classes.classification.Classification ob...,e
Clothing,"Model aspect ""clothing type""",Clothing,<odym.classes.classification.Classification ob...,c


In [103]:
Dyn_MFA_System = msc.MFAsystem(
    Name='Clothing Cycle in Households', 
    Geogr_Scope='Denmark', 
    Unit='t', 
    ProcessList=[], 
    FlowDict={}, 
    StockDict={},
    ParameterDict={}, 
    Time_Start=Model_Time_Start, 
    Time_End=Model_Time_End, 
    IndexTable=IndexTable, 
    Elements=IndexTable.loc['Element'].Classification.Items
)

In [104]:
# Start with empty process list, only process numbers (IDs) and names are needed.
Dyn_MFA_System.ProcessList = [] 
Dyn_MFA_System.ProcessList.append(msc.Process(Name = 'Environment' , ID   = 0))
Dyn_MFA_System.ProcessList.append(msc.Process(Name = 'Store'    , ID   = 1))
Dyn_MFA_System.ProcessList.append(msc.Process(Name = 'Consumer'  , ID   = 2))
Dyn_MFA_System.ProcessList.append(msc.Process(Name = 'Sorting Facility'   , ID   = 3))
Dyn_MFA_System.ProcessList.append(msc.Process(Name = 'Recycling Facility'   , ID   = 4))

# Print list of processes:
Dyn_MFA_System.ProcessList

In [108]:
Par_Recycling_Share = msc.Parameter(0.5)
Par_Reuse_Share = msc.Parameter(0.1)

Par_Clothing_Composition = df_composition[df_composition['Clothing type'].isin(clothing_type)][fibers].to_numpy()
Par_Clothing_Category_Share = df_composition[df_composition['Clothing type'].isin(clothing_type)]['Category share'].to_numpy()
Par_Clothing_Production = df_production_quarterly.to_numpy()
Par_Clothing_Survival = survival_matrix

In [110]:

ParameterDict = {}

ParameterDict['F_0_1']  = msc.Parameter(Name = 'Inflow_Clothing', ID = 1, P_Res = 1,
                                        MetaData = None, Indices = 'tec',
                                        Values = np.einsum("ce,c,ct->cet",Par_Clothing_Composition,Par_Clothing_Category_Share,Par_Clothing_Production), 
                                        Unit = 't/yr')
ParameterDict['Survival_Rate'] = msc.Parameter(Name = 'Survival Rate', ID = 2, P_Res = 2,
                                        MetaData = None, Indices = 'ca',
                                        Values = Par_Clothing_Survival, 
                                        Unit = '1/yr')
ParameterDict['Gamma_1']  = msc.Parameter(Name = 'Share of recyclable clothing', ID = 3, P_Res = 3,
                                        MetaData = None, Indices = 'c',
                                        Values = Par_Recycling_Share, 
                                        Unit = '1')
ParameterDict['Gamma_2']  = msc.Parameter(Name = 'Share of reusable clothing', ID = 4, P_Res = 3,
                                        MetaData = None, Indices = 'c',
                                        Values = Par_Reuse_Share, 
                                        Unit = '1')

In [ ]:
Dyn_MFA_System.FlowDict['F_0_1'] = msc.Flow(Name = 'Clothing Production', P_Start = 0, P_End = 1,
                                            Indices = 't,e,c', Values=None)
Dyn_MFA_System.FlowDict['F_1_2'] = msc.Flow(Name = 'Clothing Sale', P_Start = 1, P_End = 2,
                                            Indices = 't,e,c', Values=None)
Dyn_MFA_System.FlowDict['F_2_3'] = msc.Flow(Name = 'Clothing Discard', P_Start = 2, P_End = 3,
                                            Indices = 't,e,c', Values=None)
Dyn_MFA_System.FlowDict['F_3_4'] = msc.Flow(Name = 'Clothing Recycling', P_Start = 3, P_End = 4,
                                            Indices = 't,e,c', Values=None)
Dyn_MFA_System.FlowDict['F_3_1'] = msc.Flow(Name = 'Clothing Reuse', P_Start = 3, P_End = 1,
                                            Indices = 't,e,c', Values=None)
Dyn_MFA_System.FlowDict['F_3_0'] = msc.Flow(Name = 'Clothing Incineration', P_Start = 3, P_End = 0,
                                            Indices = 't,e', Values=None)
Dyn_MFA_System.FlowDict['F_4_0'] = msc.Flow(Name = 'Recycled Fibers', P_Start = 4, P_End = 0,
                                            Indices = 't,e', Values=None)
                                            
Dyn_MFA_System.StockDict['S_2']  = msc.Stock(Name = 'In-use stock', P_Res = 2, Type = 0,
                                            Indices = 't,a,e,c', Values=None, Uncert=None,
                                            ID = None, UUID = None)

Dyn_MFA_System.StockDict['dS_2']  = msc.Stock(Name = 'Net in-use stock change', P_Res = 2, Type = 1,
                                               Indices = 't,e,c', Values=None, Uncert=None,
                                               ID = None, UUID = None)

Dyn_MFA_System.Initialize_StockValues()
Dyn_MFA_System.Initialize_FlowValues()